<a href="https://colab.research.google.com/github/migostro/laboratorio-de-visao/blob/main/2_1_Rescale.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports

In [ ]:
# utilities
from operator import truediv
import os
import csv
import time
import numpy as np
from google.colab import files, drive

# image processing
from scipy import ndimage
from skimage import io, transform, filters

# visualization
from matplotlib import pyplot as plt
from mpl_toolkits.axes_grid1 import ImageGrid

# Helper Functions

In [ ]:
def display_single(image):
    fig, ax = plt.subplots()
    ax = plt.imshow(image, cmap=plt.cm.gray)
    plt.show()
    plt.close()

In [ ]:
def float_to_int(img):
    return (img*255).astype(np.uint8)

In [ ]:
def list_filepaths(path, file_extension, filepaths = []):
    if os.path.isdir(path):
        for filename in os.listdir(path):
            filepath = os.path.join(path, filename)
            if os.path.isfile(filepath) and file_extension in filename: filepaths.append(filepath) # Adiciona apenas caminhos que são de arquivos (que no nosso caso são imagens)
            else: list_filepaths(filepath, file_extension, filepaths)

    return filepaths

# Pipeline

In [ ]:
# 0 FILE READING
drive.mount('/content/drive')
root = '/content/drive/MyDrive/Colab/MAC0417/Trabalho/Versão Final'
originalData_path = root + '/originalDataSet'
resizedData_path = root + '/originalDataSet_resized'

truth_path = root + "/groundTruth"
resizedTruth_path = root + "/groundTruth_resized"

segmentedData_path = root + '/segmented_resized'

original_filepaths = list_filepaths(originalData_path, 'JPG', [])
truth_filepaths = list_filepaths(truth_path, 'png', [])
resizedTruth_filepaths = list_filepaths(resizedTruth_path, 'png', [])
segmented_filepaths = list_filepaths(segmentedData_path, 'png', [])
print(f'# of images @ {originalData_path}: {len(original_filepaths)}')
print(f'# of images @ {resizedTruth_path}: {len(resizedTruth_filepaths)}')
print(f'# of images @ {segmentedData_path}: {len(segmented_filepaths)}')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
# of images @ /content/drive/MyDrive/Colab/MAC0417/Trabalho/0_new/originalDataSet: 1080
# of images @ /content/drive/MyDrive/Colab/MAC0417/Trabalho/0_new/groundTruth_resized: 0
# of images @ /content/drive/MyDrive/Colab/MAC0417/Trabalho/0_new/segmented_resized: 1080


## Redimensionamento das imagens
A fim de reduzir os custos computacionais da segmentação e classificação, reduzimos as imagens em 80% (fator 0.2), obtendo imagens de resolução 922 x 691

Os ground truths são re-binarizados, utilizando isodata, para garantir correto funcionamento das próximas fases.

In [ ]:
# 1 RESIZE + BINARIZE IMAGES
for idx, img_path in enumerate(truth_filepaths):
    print(f'[{idx}]', img_path)
    # read
    img = io.imread(img_path)[:,:,0]
    filename = img_path.split('/')[-1]
    middle_path = img_path.replace(truth_path, '').replace(filename, '')
    
    # rescale
    img = transform.rescale(img, 0.2, anti_aliasing=False)
    # binarize
    thresh = filters.threshold_isodata(img)
    binary = ((img > thresh) * 1).astype(np.uint8)

    # write
    if not os.path.isdir(resizedTruth_path + middle_path): os.makedirs(resizedTruth_path + middle_path)
    plt.imsave(img_path.replace(truth_path, resizedTruth_path), binary, cmap=plt.cm.gray)
    # io.imsave(img_path.replace(truth_path, resizedTruth_path), binary)

[0] /content/drive/MyDrive/Colab/MAC0417/Trabalho/0_new/groundTruth/dia/interior/verde/borrachas/borrachas_001/IMG_2169.png
[1] /content/drive/MyDrive/Colab/MAC0417/Trabalho/0_new/groundTruth/dia/interior/verde/borrachas/borrachas_002/IMG_0958.png
[2] /content/drive/MyDrive/Colab/MAC0417/Trabalho/0_new/groundTruth/dia/interior/verde/borrachas/borrachas_003/IMG_0961.png
[3] /content/drive/MyDrive/Colab/MAC0417/Trabalho/0_new/groundTruth/dia/interior/verde/cartas/cartas_001/IMG_0982.png
[4] /content/drive/MyDrive/Colab/MAC0417/Trabalho/0_new/groundTruth/dia/interior/verde/copos/copos_003/IMG_0964.png
[5] /content/drive/MyDrive/Colab/MAC0417/Trabalho/0_new/groundTruth/dia/interior/verde/copos/copos_001/IMG_0967.png
[6] /content/drive/MyDrive/Colab/MAC0417/Trabalho/0_new/groundTruth/dia/interior/verde/copos/copos_002/IMG_0970.png
[7] /content/drive/MyDrive/Colab/MAC0417/Trabalho/0_new/groundTruth/dia/interior/verde/dados/dados_001/IMG_0953.png
[8] /content/drive/MyDrive/Colab/MAC0417/Traba

In [ ]:
# 2 RESIZE IMAGES
for idx, img_path in enumerate(original_filepaths):
    save_path = img_path.replace(originalData_path, resizedData_path).replace('JPG', 'png')
    print(f'[{idx}]', save_path)

    # read
    img = io.imread(img_path)
    filename = img_path.split('/')[-1]
    middle_path = img_path.replace(originalData_path, '').replace(filename, '')
    
    # rescale
    img = transform.rescale(img, 0.2, anti_aliasing=True)

    # write
    if not os.path.isdir(resizedData_path + middle_path): os.makedirs(resizedData_path + middle_path)
    io.imsave(save_path, float_to_int(img))

In [ ]:
# organize images by OBJ ID
for idx, img_path in enumerate(original_filepaths):
    print(f'[{idx}]')
    save_path = img_path.replace(originalData_path, truth_path).replace('JPG', 'png')
    original_path = save_path.replace(f"{save_path.split('/')[-2]}/", '')
    filename = save_path.split('/')[-1]

    if os.path.isfile(original_path):
        print(f'[{idx}]', save_path)
        if not os.path.isdir(save_path.replace(filename, '')): os.makedirs(save_path.replace(filename, ''))
        os.replace(original_path, save_path)